Marcin Wardyński  
czwartek, 8:00

## Lab 9: Kontekstowe odpowiadanie na pytania

Dla przygotowania danych treningowych i walidacyjnych dla modelu trzeba przerobić istniejące pliki json. Format nowych plików musi zawierać elementy:
- `id`
- `context`
- `question`
- `answers` z polem `text` zawierającym w postaci listy poprawne odpowiedzi

Poniższa funkcja dokonuje oczekiwanego przekształcenia na zbiorze PoQuAD: 

In [13]:
import json

def convert_data(data):
    results = []
    i = 0
    for article in data.get("data", []):
        for paragraph in article.get("paragraphs", []):
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                answers = qa['answers'] if 'answers' in qa.keys() else qa['plausible_answers']
                for answer in answers:
                    i += 1
                    results.append({
                        "id": i,
                        "context": context,
                        "question": question,
                        "answers": {
                            "text": [answer["generative_answer"]]
                        }
                    })
    return results


def convert_format(input_filepath, output_filepath):
    with open(input_filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    output_data = convert_data(data)
    output_wrapped_data = {"version": "0.1.0", "data": output_data}

    with open(output_filepath, "w", encoding="utf-8") as f:
        json.dump(output_wrapped_data, f, ensure_ascii=False, indent=2)


Wywołuję powyższą funkcję i zapisuję przekształcone dane w plikach `poquad-conv-train.json` oraz `poquad-conv-dev.json` dla zbiorów treningowego i walidacyjnego.

In [46]:
convert_format("poquad-train.json", "poquad-conv-train.json")
convert_format("poquad-dev.json", "poquad-conv-dev.json")

python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --dataset_name clarin-pl/poquad \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_extr

python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --train_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-train.json \
  --validation_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_abstr

python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --train_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-train.json \
  --validation_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 16 \
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_2e-5_b16

```
***** train metrics *****
  epoch                    =                3.0
  total_flos               =         86256742GF
  train_loss               =             1.6838
  train_runtime            = 2 days, 0:55:20.98
  train_samples            =              56618
  train_samples_per_second =              0.964
  train_steps_per_second   =               0.06

***** eval metrics *****
  epoch                   =        3.0
  eval_loss               =     0.9395
  eval_runtime            = 0:06:16.18
  eval_samples            =       7539
  eval_samples_per_second =      20.04
  eval_steps_per_second   =      2.507
```

python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --train_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-train.json \
  --validation_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-dev-short.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 16 \
  --learning_rate 4e-5 \
  --num_train_epochs 3 \
  --eval_strategy steps \
  --eval_steps 1 \
  --save_steps 500 \
  --metric_for_best_model f1 \
  --predict_with_generate True \
  --greater_is_better True \
  --load_best_model_at_end True \
  --save_total_limit 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_f1_4e-5_b16

python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --train_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-train.json \
  --validation_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 16 \
  --learning_rate 4e-5 \
  --num_train_epochs 3 \
  --eval_strategy steps \
  --eval_steps 500 \
  --save_steps 500 \
  --predict_with_generate True \
  --metric_for_best_model f1 \
  --greater_is_better True \
  --load_best_model_at_end True \
  --save_total_limit 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_f1_4e-5_b16

***** train metrics *****
  epoch                    =        3.0
  total_flos               = 86256742GF
  train_loss               =     0.2784
  train_runtime            = 8:25:03.53
  train_samples            =      56618
  train_samples_per_second =      5.605
  train_steps_per_second   =       0.35

***** eval metrics *****
  epoch                   =        3.0
  eval_exact_match        =    51.3881
  eval_f1                 =    67.2528
  eval_loss               =     0.8218
  eval_runtime            = 0:39:08.28
  eval_samples            =       7539
  eval_samples_per_second =       3.21
  eval_steps_per_second   =      0.402

python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --train_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-train.json \
  --validation_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 8 \
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --eval_strategy steps \
  --eval_steps 1000 \
  --save_steps 1000 \
  --predict_with_generate True \
  --metric_for_best_model f1 \
  --greater_is_better True \
  --load_best_model_at_end True \
  --save_total_limit 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_f1_2e-5_b8

***** train metrics *****
  epoch                    =         3.0
  total_flos               =  86256742GF
  train_loss               =      0.3401
  train_runtime            = 13:55:17.07
  train_samples            =       56618
  train_samples_per_second =       3.389
  train_steps_per_second   =       0.424

***** eval metrics *****
  epoch                   =        3.0
  eval_exact_match        =    51.5297
  eval_f1                 =    67.6208
  eval_loss               =     0.8603
  eval_runtime            = 0:41:46.65
  eval_samples            =       7539
  eval_samples_per_second =      3.008
  eval_steps_per_second   =      0.376

python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --train_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-train.json \
  --validation_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 16 \
  --learning_rate 5e-5 \
  --num_train_epochs 3 \
  --eval_strategy steps \
  --eval_steps 500 \
  --save_steps 500 \
  --predict_with_generate True \
  --metric_for_best_model f1 \
  --greater_is_better True \
  --load_best_model_at_end True \
  --save_total_limit 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_f1_5e-5_b16

```
***** train metrics *****
  epoch                    =         3.0
  total_flos               =  86256742GF
  train_loss               =      0.3226
  train_runtime            = 13:12:21.12
  train_samples            =       56618
  train_samples_per_second =       3.573
  train_steps_per_second   =       0.223

***** eval metrics *****
  epoch                   =        3.0
  eval_exact_match        =    52.3513
  eval_f1                 =    68.3034
  eval_loss               =     0.8013
  eval_runtime            = 0:39:11.65
  eval_samples            =       7539
  eval_samples_per_second =      3.206
  eval_steps_per_second   =      0.401
```



python run_seq2seq_qa.py \
  --model_name_or_path allegro/plt5-base \
  --train_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-train.json \
  --validation_file /Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab9/poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 3 \
  --eval_strategy steps \
  --eval_steps 500 \
  --save_steps 500 \
  --predict_with_generate True \
  --metric_for_best_model f1 \
  --greater_is_better True \
  --load_best_model_at_end True \
  --save_total_limit 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_f1_3e-5_b12

```
***** train metrics *****
  epoch                    =               3.0
  total_flos               =        86256742GF
  train_loss               =            1.2486
  train_runtime            = 1 day, 4:45:16.29
  train_samples            =             56618
  train_samples_per_second =             1.641
  train_steps_per_second   =             0.137

***** eval metrics *****
  epoch                   =        3.0
  eval_exact_match        =    50.8499
  eval_f1                 =    66.8706
  eval_loss               =     0.8446
  eval_runtime            = 0:47:32.25
  eval_samples            =       7539
  eval_samples_per_second =      2.643
  eval_steps_per_second   =      0.331
```

In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

ready_model_name = "apohllo/plt5-base-poquad"
ready_model_tokenizer = AutoTokenizer.from_pretrained(ready_model_name)
ready_model = AutoModelForSeq2SeqLM.from_pretrained(ready_model_name)

In [15]:
context = "Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3. § 2. Jeżeli sprawca dopuszcza się czynnej napaści w związku z pełnieniem przez przełożonego obowiązków służbowych albo wspólnie z innymi żołnierzami lub w obecności zebranych żołnierzy, podlega karze pozbawienia wolności od 6 miesięcy do lat 8. § 3. Jeżeli sprawca czynu określonego w § 1 lub 2 używa broni, noża lub innego podobnie niebezpiecznego przedmiotu, podlega karze pozbawienia wolności od roku do lat 10. § 4. Karze przewidzianej w § 3 podlega sprawca czynu określonego w § 1 lub 2, jeżeli jego następstwem jest skutek określony w art. 156 lub 157 § 1."
question = "Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?"

input_text = f"question: {question} context: {context}"

inputs = ready_model_tokenizer(input_text, return_tensors="pt")

outputs = ready_model.generate(inputs["input_ids"], max_length=50, num_beams=5, early_stopping=True)

answer = ready_model_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?
Answer: tak


Do fine-tuningu swojego modelu użyłem nieznacznie zmienionego skryptu `run_seq2seq_qa.py` z repozytorium `transformers`, który został wywołany z następującymi parametrami:
```
  --model_name_or_path allegro/plt5-base \
  --train_file ../../../..poquad-conv-train.json \
  --validation_file ../../../..poquad-conv-dev.json \
  --context_column context \
  --question_column question \
  --answer_column answers \
  --do_train \
  --do_eval \
  --per_device_train_batch_size X \
  --learning_rate Ye-5 \
  --num_train_epochs 3 \
  --eval_strategy steps \
  --eval_steps 500 \
  --save_steps 500 \
  --predict_with_generate True \
  --metric_for_best_model f1 \
  --greater_is_better True \
  --load_best_model_at_end True \
  --save_total_limit 3 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ../../../../model_poquad_t5_base_f1_3e-5_b12
```

Jak widzimy powyżęj, jako modelu pretrenowanego używam `allegro/plt5-base`, gdyż moja konfiguracja komputera nie pozwala mi na wykorzystanie pojemniejszego modelu. Do treningu przekazuję uprzednio przygotowane pliki ze zbiorem danych PoQuAD oraz wskazuje nazwy odpowiednich kolumn. Zlecam wykonanie treningu i ewaluacji z wielkością batcha i krokiem treningu zdefiniowanym dla każdego uruchomienia skryptu z innymi wartościami. Trening ma trwać zalecane trzy epoki, a wybrana strategia ewaluacji `f1` powinna zostać uruchomiona co 500 kroków. Finalnie powinien zostać zaladowany model o najlepszym wyniku ewaluacji. Parametry `max_seq_length` oraz `doc_stride` otrzymały wartości odpowiednio 384 i 128. (Chociaż pierwsza z nich mogłaby w zasadzie otrzymać wartość 512).

Wyniki skyptu dla wybranych parametrów treningowych:

| lr   | #batch-y | eval em | eval f1 |
|------|----------|---------|---------|
| 2e-5 | 8        | 51.53   | 67.62   |
| 3e-5 | 12       | 50.84   | 66.87   |
| 4e-5 | 16       | 51.39   | 67.25   |
| 5e-5 | 16       | 52.35   | 68.30   |

Jak widzimy, dla danych walidacyjnych zbioru PoQuAD najlepiej wypada model o stałej uczącej `5e-5` i rozmiarze batch-a `16` i to zarówno dla metryki *Exact Match*, jak i *f1*.

Skoro mamy już wytrenowany model wraz z jego wstępną ewaluacją, skupmy się na danych testowych. Poniższy zestaw funkcji buduje struktury słownikowe, które ułatwiają nawigowanie po danych testowych zawierających wskazane pytania prawne:

In [11]:
import json

NO_ANS = "no_ans"

class QA:
    def __init__(self, question_id, question, answer):
        self.question_id = question_id
        self.question = question
        self.answer = answer

class Entry:
    def __init__(self, passage_id, passage_text, qas):
        self.passage_id = passage_id
        self.passage_text = passage_text
        self.qas = qas

def init_qas_with_answers(filepath):
    qa_dict = {}
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "score" in record and record["score"] == "1"\
                    and "question-id" in record and "answer" in record:
                qa = QA(record["question-id"], None, record["answer"])
                qa_dict[record["question-id"]] = qa
    return qa_dict

def match_questions_to_answers(filepath, qa_dict):
    q_wo_a = []
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "text" in record and "_id" in record:
                if record["_id"] in qa_dict.keys():
                    qa = qa_dict[record["_id"]]
                    qa.question = record["text"]
                else:
                    qa_dict[NO_ANS].append(record["text"])


def organize_question_to_context_relations(filepath, qa_dict):
    qc_dict = {}
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "score" in record and record["score"] == "1"\
                    and "passage-id" in record\
                    and "question-id" in record and record["question-id"] in qa_dict.keys():
                if record["passage-id"] not in qc_dict.keys():
                    qc_dict[record["passage-id"]] = []
                qc_dict[record["passage-id"]].append(record["question-id"])
    return qc_dict

def load_passages(filepath, qc_dict, qa_dict):
    entries = []
    with open(filepath, "r") as file:
        for line in file:
            record = json.loads(line.strip())
            
            if "text" in record and "_id" in record and record["_id"] in qc_dict.keys():
                qa_ids = qc_dict[record["_id"]]
                qas = []
                for qa_id in qa_ids:
                    qas.append(qa_dict[qa_id])
                entries.append(Entry(record["_id"], record["text"], qas))
    return entries

Po uruchomieniu tych fynkcji i przekazaniu odpowiednich plików wejściowych otrzymujemy następujące struktury:
`qa_dict` - słownik łączący pytania z odpowiedziami
`qc_dict` - słownik łączący pytania z kontekstem
`test_passages` - lista z kontekstem oraz połączonymi z nim pytaniami i odpowiedziami

Tak przygotowanych struktur będą wykorzystywać, aby zewaluować jakość modelu dla danych testowych.

In [2]:
qa_dict = init_qas_with_answers("simple-legal-questions-pl-main/answers.jl")

qa_dict[NO_ANS] = []
match_questions_to_answers("simple-legal-questions-pl-main/questions.jl", qa_dict)

qc_dict = organize_question_to_context_relations("simple-legal-questions-pl-main/relevant.jl", qa_dict)
test_passages = load_passages("simple-legal-questions-pl-main/passages.jl", qc_dict, qa_dict)

Dodatkowo napisałem funkcję przekształcającą dane zbioru PoQuAD do reprezentacji odpowiadającej danym testowym. Napisałem ją, gdyż chciałbym dodatkowo policzyć metryki dla zbioru walidacyjnego dokładnie tymi samymi funkcjami, co dla zbioru testowego.

In [3]:
def convert_poquad_data(filepath):
    val_entries = []
    with open(filepath, "r") as file:
        json_content = json.load(file)
        for data in json_content['data']:
            for paragraph in data['paragraphs']:
                qa_list = []
                for qa in paragraph['qas']:
                    answers = qa['answers'] if 'answers' in qa.keys() else qa['plausible_answers']
                    for answer in answers:
                        qa_obj = QA(None, qa['question'], answer['generative_answer'])
                        qa_list.append(qa_obj)
                    
                entry = Entry(None, paragraph['context'], qa_list)
                val_entries.append(entry)
    return val_entries

val_passages = convert_poquad_data("poquad-dev.json")

Funkcja `exec_passages` przechodzi po kontekstach w podanym zbiorze i przekazuje je do modelu wraz z pytaniami i odpowiedziami na nie. Wyjściowo otrzymujemy listę odpowiedzi modelu oraz oczekiwanych odpowiedzi generatywnych.

In [4]:
from tqdm import tqdm

def exec_passages(model, tokenizer, passages):
    answers = []
    expected_answers = []
    for passage in tqdm(passages):
        for qa in passage.qas:
            input_text = f"question: {qa.question} context: {passage.passage_text}"
            inputs = tokenizer(input_text, return_tensors="pt")

            outputs = model.generate(inputs["input_ids"], max_length=100, num_beams=5, early_stopping=True)

            answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
            answers.append(answer)
            expected_answers.append(qa.answer)
    return answers, expected_answers


Teraz wystarczy wczytać odpowiedni model i wyznaczyć jego odpowiedzi na pytania ze zbioru poprzez wywolanie powyższej funkcji:

In [24]:
answers, expected_answers = exec_passages(ready_model, ready_model_tokenizer, test_passages)

100%|██████████| 557/557 [16:48<00:00,  1.81s/it]


In [73]:
val_answers, val_expected_answers = exec_passages(ready_model, ready_model_tokenizer, val_passages)

  4%|▍         | 55/1453 [04:46<2:01:27,  5.21s/it]


KeyboardInterrupt: 

In [93]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

my_t5_base_model_name = "./model_poquad_abstr"
my_t5_base_model_tokenizer = AutoTokenizer.from_pretrained(my_t5_base_model_name)
my_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(my_t5_base_model_name)

In [96]:
my_answers, expected_answers = exec_passages(my_t5_base_model, my_t5_base_model_tokenizer, test_passages)

100%|██████████| 557/557 [11:03<00:00,  1.19s/it]


In [72]:
my_val_answers, val_expected_answers = exec_passages(my_t5_base_model, my_t5_base_model_tokenizer, val_passages)

100%|██████████| 1453/1453 [2:33:27<00:00,  6.34s/it]  


In [97]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

lr2e_5_b16_t5_base_model_name = "./model_poquad_t5_base_2e-5_b16"
lr2e_5_b16_t5_base_model_tokenizer = AutoTokenizer.from_pretrained(lr2e_5_b16_t5_base_model_name)
lr2e_5_b16_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(lr2e_5_b16_t5_base_model_name)

In [98]:
lr2e_5_b16_t5_base_answers, expected_answers = exec_passages(lr2e_5_b16_t5_base_model, lr2e_5_b16_t5_base_model_tokenizer, test_passages)

100%|██████████| 557/557 [13:39<00:00,  1.47s/it]


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

lr4e_5_b16_t5_base_model_name = "./model_poquad_t5_base_f1_4e-5_b16"
lr4e_5_b16_t5_base_model_tokenizer = AutoTokenizer.from_pretrained(lr4e_5_b16_t5_base_model_name)
lr4e_5_b16_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(lr4e_5_b16_t5_base_model_name)

In [7]:
lr4e_5_b16_t5_base_answers, expected_answers = exec_passages(lr4e_5_b16_t5_base_model, lr4e_5_b16_t5_base_model_tokenizer, test_passages)

100%|██████████| 557/557 [16:26<00:00,  1.77s/it]


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

lr2e_5_b8_t5_base_model_name = "./model_poquad_t5_base_f1_2e-5_b8"
lr2e_5_b8_t5_base_model_tokenizer = AutoTokenizer.from_pretrained(lr2e_5_b8_t5_base_model_name)
lr2e_5_b8_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(lr2e_5_b8_t5_base_model_name)

In [ ]:
lr2e_5_b8_t5_base_answers, expected_answers = exec_passages(lr2e_5_b8_t5_base_model, lr2e_5_b8_t5_base_model_tokenizer, test_passages)

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

lr5e_5_b16_t5_base_model_name = "./model_poquad_t5_base_f1_5e-5_b16"
lr5e_5_b16_t5_base_model_tokenizer = AutoTokenizer.from_pretrained(lr5e_5_b16_t5_base_model_name)
lr5e_5_b16_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(lr5e_5_b16_t5_base_model_name)

In [7]:
lr5e_5_b16_t5_base_answers, expected_answers = exec_passages(lr5e_5_b16_t5_base_model, lr5e_5_b16_t5_base_model_tokenizer, test_passages)

100%|██████████| 557/557 [14:17<00:00,  1.54s/it]


In [15]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

lr3e_5_b12_t5_base_model_name = "./model_poquad_t5_base_f1_3e-5_b12"
lr3e_5_b12_t5_base_model_tokenizer = AutoTokenizer.from_pretrained(lr3e_5_b12_t5_base_model_name)
lr3e_5_b12_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(lr3e_5_b12_t5_base_model_name)

In [16]:
lr3e_5_b12_t5_base_answers, expected_answers = exec_passages(lr3e_5_b12_t5_base_model, lr3e_5_b12_t5_base_model_tokenizer, test_passages)

100%|██████████| 557/557 [13:42<00:00,  1.48s/it]


Poniżej znajduje się funkcja odnajdująca dokładne dopasowanie (*Exact Match*). Zawiera ona mały etap wstępnego przetworzenia tekstów na wejściu, który to usuwa skrajne spacje, znaki interpunkcyjne i zmniejsza czcionkę wszystkich słów, tak żeby lepiej odkrywać dokładne dopasowania.

In [22]:
import string

def clean_text(s):
    s = s.strip()
    s = s.lower()
    s = ''.join(c for c in s if c not in string.punctuation)
    return s


def calculate_exact_matches(answers, expected_answers, clean_fun):
    matches = 0
    for s1, s2 in zip(answers, expected_answers):
        if clean_fun(s1) == clean_fun(s2):
            matches += 1
    return matches/len(answers)

In [107]:
calculate_exact_matches(my_answers, expected_answers, clean_text)

0.2456445993031359

In [106]:
calculate_exact_matches(lr2e_5_b16_t5_base_answers, expected_answers, clean_text)

0.2264808362369338

In [9]:
calculate_exact_matches(lr4e_5_b16_t5_base_answers, expected_answers, clean_text)

0.24390243902439024

In [105]:
calculate_exact_matches(answers, expected_answers, clean_text)

0.2456445993031359

In [14]:
calculate_exact_matches(lr5e_5_b16_t5_base_answers, expected_answers, clean_text)

0.24738675958188153

In [23]:
calculate_exact_matches(lr3e_5_b12_t5_base_answers, expected_answers, clean_text)

0.2874564459930314

I ostatecznie funkcja mierząca wartości TP, FP, FN i wyznaczająca na ich podstawie metryki `precision` i `recall` aby zwrócić opierającą się o nie metrykę `f1`. Poza dwoma odpowiedziami funkcja przyjmuje funkcję tokenizującą, która przeprowadza czyszczenie tekstu takie samo, jak w przypadku *Exact Match*, i dodatkowo tworzy tokeny z już dalej nieprzetworzonych słów zdania.

In [24]:
from collections import Counter
import re

def tokenize(text):
    text = clean_text(text)
    tokens = re.split(r"[^\w]+", text)
    return tokens

def compute_single_f1(tokens1, tokens2):
    TP = sum((tokens1 & tokens2).values())
    FP = sum((tokens1 - tokens2).values())
    FN = sum((tokens2 - tokens1).values())

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

def compute_f1(answers, expected_answers, tokenize_fun):

    f1_scores = []
    for s1, s2 in zip(answers, expected_answers):
        s1_t = Counter(tokenize_fun(s1))
        s2_t = Counter(tokenize_fun(s2))
        f1 = compute_single_f1(s1_t, s2_t)
        f1_scores.append(f1)

    return sum(f1_scores)/len(f1_scores)


In [102]:
compute_f1(answers, expected_answers, tokenize)

0.5184004475490456

In [103]:
compute_f1(my_answers, expected_answers, tokenize)

0.5120417923766273

In [104]:
compute_f1(lr2e_5_b16_t5_base_answers, expected_answers, tokenize)

0.5053607657978532

In [11]:
compute_f1(lr4e_5_b16_t5_base_answers, expected_answers, tokenize)

0.5172354390737379

In [10]:
compute_f1(lr5e_5_b16_t5_base_answers, expected_answers, tokenize)

0.5125987055942087

In [25]:
compute_f1(lr3e_5_b12_t5_base_answers, expected_answers, tokenize)

0.5257402393193848

In [90]:
compute_f1(my_val_answers, val_expected_answers, tokenize)


0.7216107340920863

Żeby sprawdzić, czy fleksja ma duże znaczenie przy obliczaniu metryki `f1`, stworzę dodatkową funkcję, która poza wyodrębnieniem poszczególnych słów w odpowidzi, dodatkowo zlematyzuje te słowa.

In [27]:
!python -m spacy download pl_core_news_lg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.7/573.7 MB 3.6 MB/s eta 0:00:0000:0100:05
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


In [ ]:
import spacy

nlp = spacy.load("pl_core_news_lg")

def tokenize_and_lemmatize(text):
    tokens = tokenize(text)
    text = " ".join(tokens)

    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

In [31]:
compute_f1(lr3e_5_b12_t5_base_answers, expected_answers, tokenize_and_lemmatize)

0.5395675134388546

Okazuje się, że dodatkowa lematyzacja słów dla `f1` tylko nieznacznie poprawia wynik, co jest dla mnie poniekąd zaskoczeniem.

Mimo wszystko języki z bogatą fleksją już na poziomie przygotowywania modelu wpływają na jego działanie. Mianowicie model ten musi uwzględnić fleksję zarówno przy tokenizacji, jak i przy budowaniu osadzeń dla wczytanych tokenów. Fleksja prowadzi najpewniej do zwiększenia liczby unikalnych tokenów obsługiwanych przez model, oraz musi zostać uwzględniona w budowie kontekstu dla poszczególnych tokenów, przez co potrzebujemy pojemniejszego modelu i większej liczby zdań w korpusie uczącym, aby dobrze odwzorować zachodzące zależności fleksyjne.

Takie dodatkowe wymagania wobec modelu utrudniają jego działanie i przyczyniają się do pogorszenia jakości wyników względem porównywalnego zbioru danych w języku o uboższej fleksji.

In [33]:
import json

def store_answers_in_json(answers, file_path):
    with open(file_path, "w") as f:
        json.dump(answers, f)

def load_answers_from_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

In [34]:
store_answers_in_json(lr3e_5_b12_t5_base_answers, "lr3e_5_b12_t5_base_answers.json")